In [1]:
from Helpers import helpers

/opt/conda/lib/python3.9/site-packages/art/estimators/certification/__init__.py:14: UserWarning: PyTorch not found. Not importing DeepZ or Interval Bound Propagation functionality
  warnings.warn("PyTorch not found. Not importing DeepZ or Interval Bound Propagation functionality")


In [2]:
from sklearn.datasets import make_blobs

#n_samples = 2000
#X, y_true = make_blobs(n_samples=n_samples, centers=4, n_features=3,
#                       cluster_std=0.60, random_state=0)
#X.shape

In [3]:
#_, perturbed_df = helpers.load_plain_and_perturbed_dataset(3, import_path='./export/plain.csv', perturbed_path='./export/truncated/')
seeds_dataset = helpers.load_dataset('../data/heart-dataset/heart_numerical.csv')
seeds_dataset.dropna(inplace=True)
X = seeds_dataset.drop('class', axis=1)
y_true = seeds_dataset['class']


In [4]:
import numpy as np


X[X.isna()].count()


baseline value               0
accelerations                0
fetal_movement               0
uterine_contractions         0
light_decelerations          0
histogram_width              0
histogram_min                0
histogram_max                0
histogram_number_of_peaks    0
dtype: int64

In [5]:
y_true.shape

(2126,)

In [6]:
from Helpers import threed_laplace
X.columns
X_3d = X[['baseline value', 'accelerations', 'histogram_min']]

In [7]:
test_df = helpers.run_mi_experiments(X_3d.values, y_true.values, helpers.get_experiment_epsilons(), targets=3, n_times=10, columns=['A', 'P', 'WK'], algorithm=threed_laplace.generate_truncated_perturbed_dataset)

0.6308851224105462 0.5319548872180451
0.608286252354049 0.5112781954887218
0.6064030131826742 0.5545112781954887
0.6760828625235404 0.5883458646616542
0.6101694915254238 0.5225563909774437
0.6271186440677966 0.6052631578947368
0.7005649717514124 0.6578947368421053
0.5856873822975518 0.4868421052631579
0.632768361581921 0.5977443609022557
0.7099811676082862 0.6203007518796992
0.6930320150659134 0.6635338345864662
0.5668549905838042 0.5469924812030075
0.7702448210922788 0.706766917293233
0.8041431261770244 0.7236842105263158
0.6440677966101694 0.575187969924812
0.743879472693032 0.6823308270676691
0.6930320150659134 0.631578947368421
0.7212806026365348 0.6672932330827067
0.7231638418079096 0.6259398496240601
0.7777777777777778 0.6672932330827067
0.6704331450094162 0.6541353383458647
0.7740112994350282 0.7537593984962406
0.7231638418079096 0.6541353383458647
0.5894538606403014 0.5770676691729323
0.711864406779661 0.6071428571428571
0.7702448210922788 0.7236842105263158
0.6177024482109228 

In [8]:
test_df.groupby('epsilon').mean()

,shokri_mi_adv,attack_adv,tpr,fpr,run
epsilon,,,,,
0.1,0.071126,0.530017,0.638795,0.567669,4.5
0.5,0.064687,0.523433,0.713748,0.649060,4.5
1.0,0.048686,0.518300,0.700753,0.652068,4.5
2.0,0.050788,0.518002,0.719021,0.668233,4.5
3.0,0.042557,0.514730,0.740490,0.697932,4.5
5.0,0.058193,0.520001,0.758569,0.700376,4.5
7.0,0.053445,0.519409,0.732580,0.679135,4.5
9.0,0.051776,0.518225,0.745009,0.693233,4.5


In [14]:
test_df.to_csv('./export/privacy/report-advantages-' + str(n_samples) + '.csv')

NameError: name 'n_samples' is not defined

In [ ]:
truncated = helpers.run_mi_experiments(X, y_true, helpers.get_experiment_epsilons(), n_times=20, algorithm=generate_truncated_perturbed_dataset)

NameError: name 'generate_truncated_perturbed_dataset' is not defined

In [ ]:
truncated.to_csv('./export/privacy/truncated-report-advantages-' + str(n_samples) + '.csv')